In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import skrf as rf
#rf.stylely()
skrf_color_names = ['blue','red','green','orange','purple','pink','grey']
skrf_colors = dict(zip(skrf_color_names, rf.plotting.colors()))
import math

In [ ]:
def r_range(start, stop, num_steps):
    r = np.linspace(1, num_steps, num_steps)
    print(r)
    k = np.log(start/stop)/(1-num_steps)
    print(k)
    print(stop)
    a0 = stop*np.exp(-k*num_steps)
    print(a0)
    for i in range(num_steps):
        r[i] = a0*np.exp(k*(i+1))
        
    return r

z0 = 50
r = r_range(1, 100, 5)
r = np.linspace(0, 100, 5)
x = np.linspace(-20, 20, 5)
zr, zx = np.meshgrid(r,x)
z = zr + 1j*zx

print(np.array2string(r, precision=2, floatmode='fixed'))
gamma = np.divide(z-z0, z+z0)
#print(gamma)

rf.plotting.plot_smith(gamma, marker='o')

In [ ]:
#x = np.linspace(0,50,100)
# For gamma in a + jb format
z0 = 50
gamma = 0.5 +1j*0.0
z = z0* np.divide((1-gamma), 1.000001+gamma)
print(f'Gamma {gamma:.02f} is {z:.02f} ohms given Z0={z0:.02f}')

In [ ]:
import pwdatatools as pw
ds = pw.file_read(r'C:\Users\derek\Documents\ADS\2021u2\workspaces\CreeAsymmetricDoherty_wrk\data\Manual_LP_15W.ds')

In [ ]:
print(ds.blocks[0].ivarnames)
print(ds.blocks[0].dvarnames)v
print(ds.blocks[0].dependencies)

In [ ]:
block = ds.blocks[0]
for ivarname in block.ivarnames:
    print(f'{ivarname}')
    print(block.what(f'{ivarname}'))
    print('\n')
    
print(block.datatypes)
print(block.datanames)

In [ ]:
data = block.variables[block.variables['freq'] == 3.600000e+09]
data = data[data.columns.drop(list(data.filter(regex='bonding')))]
rho_im = data.get('rho_im').to_numpy()
rho_re = data.get('rho_re').to_numpy()

In [ ]:
plt.rcParams['figure.dpi'] = 300
fig, axes = plt.subplots()
rf.plotting.smith(ax=axes)

lp_gamma_points = gamma
#x = np.real(lp_gamma_points)
#y = np.imag(lp_gamma_points)
x = rho_re
y = rho_im
z = np.real(ds.blocks[13]['Pout_dBm'].to_numpy())

#axes.plot(rho_re,rho_im, marker='.', color=skrf_colors['blue'], linestyle='None', markersize=1)
graph = axes.scatter(x,y, c=z, s=1, marker='.', cmap='viridis', linestyle='None')
fig.colorbar(graph)
axes.axis(1*np.array([-1.1, 1.1, -1.1, 1.1]))

In [ ]:
fig, ax = plt.subplots()
rf.plotting.smith(ax=ax)

import matplotlib.tri as tri

x = rho_re
y = rho_im

# Pout
z = np.real(ds.blocks[13]['Pout_dBm'].to_numpy())

# Plot loadpull grid
ax.plot(x, y, marker='.', color="grey", markersize=1, linestyle='None', alpha=0.25)

# Plot Pout contours
z_min = np.floor(np.min(z))
z_max = np.round(np.max(z))
levels = np.linspace(z_min, z_max, int((z_max-z_min)/0.5)+1)
print(levels)
max_index = np.argmax(z)

contour1=ax.tricontour(x, y, z, levels=levels, linewidths=0.5, cmap="Blues")
#contour = ax.tricontourf(x, y, z, levels=10, cmap="RdBu_r")
ax.plot(x[max_index], y[max_index], 'bo', markersize=1)

#ax.clabel(contour, inline=True, fontsize=5)
#fig.colorbar(contour, ax=ax)

# Plot PAE contours
z2 = np.real(ds.blocks[4]['PAE'].to_numpy())
z_min = np.floor(np.min(z2))
z_max = np.round(np.max(z2))
levels = np.linspace(z_min, z_max, int((z_max-z_min)/2.5+1))
print(levels)

contour2 = ax.tricontour(x, y, z2, levels=levels, linewidths=0.5, cmap="Reds")

# Plot maximum point
max_index = np.argmax(z2)
ax.plot(x[max_index], y[max_index], 'ro', markersize=1)

#ax.clabel(contour, inline=True, fontsize=5)
#fig.colorbar(contour, ax=ax)

ax.axis(1*np.array([-1.1, 1.1, -1.1, 1.1]))

In [ ]:
for index, db in enumerate(ds.blocks):
    print(f'{index}: {db.dvarnames}')

In [ ]:
y = ds.blocks[13]['Pout_dBm']
x = ds.blocks[15]['Pin_dBm']

fig, ax = plt.subplots()
rf.plotting.plot_rectangular(x, y, ax=ax)

In [ ]:
s11 = ds.blocks[10]['S[1,1]'].to_numpy()

rf.plotting.plot_complex_rectangular(s11)